In [ ]:
import pandas as pd
from prophet import Prophet
import numpy as np
import matplotlib.pyplot as plt
from prophet.diagnostics import cross_validation, performance_metrics
import os

df = pd.read_csv('base_transacciones_final.csv')
df['fecha'] = pd.to_datetime(df['fecha'])

top_clientes = df['id'].value_counts().head(3).index.tolist()
df = df[df['id'].isin(top_clientes)]

grouped = df.groupby(['id', 'comercio'])
predicciones = []

os.makedirs("graficas_forecasting", exist_ok=True)

for (cliente, comercio), grupo in grouped:

    serie = grupo[['fecha', 'monto']].copy()
    serie = serie.rename(columns={'fecha': 'ds', 'monto': 'y'})
    serie = serie.groupby('ds').sum().reset_index()

    if serie['ds'].nunique() < 10:
        continue

    modelo = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=False)
    modelo.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    modelo.fit(serie)

    futuro = modelo.make_future_dataframe(periods=180)
    pronostico = modelo.predict(futuro)

    futuro_only = pronostico[pronostico['ds'] > serie['ds'].max()]
    pico = futuro_only.loc[futuro_only['yhat'].idxmax()]

    if pico['yhat'] < 0:
        continue

    std_y = serie['y'].std()
    if std_y == 0:
        continue
    z = (pico['yhat'] - serie['y'].mean()) / std_y
    if abs(z) > 5:
        continue

    y_mean = serie['y'].mean()

    var_total = np.var(pronostico['yhat'])
    var_daily = np.var(pronostico['daily'])
    var_weekly = np.var(pronostico['weekly'])
    var_monthly = np.var(pronostico['monthly'])

    frecuencia = serie['ds'].sort_values().diff().dropna().dt.days.value_counts()
    dias_consecutivos = frecuencia.get(1, 0)

    criterio_var = 0.5
    periodicidades = []
    var_dict = {}

    if var_weekly / var_total > criterio_var:
        periodicidades.append("Weekly")
        var_dict["Weekly"] = var_weekly
    if var_monthly / var_total > criterio_var:
        periodicidades.append("Monthly")
        var_dict["Monthly"] = var_monthly
    if var_daily / var_total > criterio_var and dias_consecutivos >= 5:
        periodicidades.append("Daily")
        var_dict["Daily"] = var_daily

    if not periodicidades:
        continue

    tiene_periodicidad = ", ".join(periodicidades)
    principal_periodicidad = max(var_dict, key=var_dict.get)

    try:
        total_days = (serie['ds'].max() - serie['ds'].min()).days
        initial_days = max(60, total_days - 60)
        initial = f"{initial_days} days"

        df_cv = cross_validation(modelo, initial=initial, period='30 days', horizon='60 days', parallel="processes")
        df_perf = performance_metrics(df_cv)
        mae = df_perf['mae'].mean()
        rmse = df_perf['rmse'].mean()
        mape = df_perf['mape'].mean()
    except:
        mae = rmse = mape = np.nan

    if mape > 15:
        continue

    predicciones.append({
        'id_cliente': cliente,
        'comercio': comercio,
        'fecha_ultima': serie['ds'].max().date(),
        'fecha_predicha': pico['ds'].date(),
        'monto_estimado': round(pico['yhat'], 2),
        'amplitud_daily': round(np.max(np.abs(pronostico['daily'])), 2),
        'amplitud_weekly': round(np.max(np.abs(pronostico['weekly'])), 2),
        'amplitud_monthly': round(np.max(np.abs(pronostico['monthly'])), 2),
        'mayor contributor': principal_periodicidad,
        'periodicidades_detectadas': tiene_periodicidad,
        'ratio_daily': var_daily / var_total,
        'ratio_weekly': var_weekly / var_total,
        'ratio_monthly': var_monthly / var_total,
        'mae': round(mae, 2),
        'rmse': round(rmse, 2),
        'mape': round(mape, 2)
    })

    safe_comercio = "".join(c if c.isalnum() else "_" for c in comercio)

    fig = modelo.plot(pronostico)
    plt.title(f"Forecast para {cliente} - {comercio}")
    plt.savefig(f"graficas_forecasting/{cliente}_{comercio}.png")
    plt.close()

    fig_comp = modelo.plot_components(pronostico)
    plt.suptitle(f"Componentes estacionales - {cliente} - {comercio}", fontsize=14)
    plt.savefig(f"graficas_forecasting/{cliente}_{safe_comercio}_componentes.png")
    plt.close()

df_resultado = pd.DataFrame(predicciones)
df_resultado.to_csv("forecasting.csv", index=False)
print(df_resultado)